# 构建神经网络

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

D:\Anaconda\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [6]:
#初始化神经网络参数
#第一层节点数
n_hidden_1 = 256
#第二层节点数
n_hidden_2 = 128
#输入层
n_input = 784
#输出层
n_classes = 10

x = tf.placeholder("float",[None,n_input])
y = tf.placeholder("float",[None,n_classes])
#初始化，高斯分布，stddev标准差
stddev = 0.1
weights = {
    'w1':tf.Variable(tf.random_normal([n_input,n_hidden_1],stddev=stddev)),
    'w2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2],stddev=stddev)),
    'out':tf.Variable(tf.random_normal([n_hidden_2,n_classes],stddev=stddev))
}
biases = {
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

print("Netword Ready!")


Netword Ready!


In [15]:
def multilayer_perceptron(_X,_weights,_biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X,_weights['w1']),_biases['b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1,_weights['w2']),_biases['b2']))
#     return (tf.matmul(layer_2,_weights['out']) + _biases['out'])
    return tf.add(tf.matmul(layer_2,_weights['out']),_biases['out'])

In [17]:
#预测
pred = multilayer_perceptron(x,weights,biases)
#loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optm = tf.train.GradientDescentOptimizer(learning_rate = 0.001).minimize(cost)
corr = tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr,"float"))

print("FUNCTIONS READY")

FUNCTIONS READY


In [19]:
training_epochs = 20
batch_size = 100
display_step = 4
#launch the graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples/batch_size)
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optm,feed_dict={x:batch_xs,y:batch_ys})
        avg_cost += sess.run(cost,feed_dict={x:batch_xs,y:batch_ys})
    avg_cost = avg_cost/total_batch
    
    if epoch % display_step == 0 :
        print("Epoch: %03d/%03d cost: %.9f"%(epoch,training_epochs,avg_cost))
        train_acc = sess.run(accr,feed_dict={x:batch_xs,y:batch_ys})
        print("TRAIN ACCURACY：%.3f"%(train_acc))
        train_acc = sess.run(accr,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print("Test ACCURACY: %.3f"%(train_acc))
print("Finsh!")

Epoch: 000/020 cost: 2.383366595
TRAIN ACCURACY：0.130
Test ACCURACY: 0.132
Epoch: 004/020 cost: 2.240368398
TRAIN ACCURACY：0.350
Test ACCURACY: 0.297
Epoch: 008/020 cost: 2.200521219
TRAIN ACCURACY：0.400
Test ACCURACY: 0.422
Epoch: 012/020 cost: 2.154432045
TRAIN ACCURACY：0.460
Test ACCURACY: 0.478
Epoch: 016/020 cost: 2.099822612
TRAIN ACCURACY：0.490
Test ACCURACY: 0.536
Finsh!


# CNN(卷积神经网络)

In [1]:
import numpy as np 
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


D:\Anaconda\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [3]:
#输入节点
n_input = 784
#输出节点数
n_outout = 10
#权重
weights = {
    #卷积层权重
    'wc1':tf.Variable(tf.random_normal([3,3,1,64],stddev=0.1)),
    #卷积层权重
    'wc2':tf.Variable(tf.random_normal([3,3,64,128],stddev=0.1)),
    #全连接层权重
    'wd1': tf.Variable(tf.random_normal([7*7*128,1024],stddev=0.1)),
    #输出层权重
    'wd2':tf.Variable(tf.random_normal([1024,n_outout],stddev=0.1))
}
biases = {
    'bc1':tf.Variable(tf.random_normal([64],stddev=0.1)),
    'bc2':tf.Variable(tf.random_normal([128],stddev=0.1)),
    'bd1':tf.Variable(tf.random_normal([1024],stddev=0.1)),
    'bd2':tf.Variable(tf.random_normal([n_outout],stddev=0.1))
}

In [4]:
def conv_basic(_input,_w,_b,_keepratio):
    #输入层
    _input_r = tf.reshape(_input,shape=[-1,28,28,1])
    #卷积层1
    _conv1 = tf.nn.conv2d(_input_r,_w['wc1'],strides=[1,1,1,1],padding="SAME")
    #激活函数
    _conv1 = tf.nn.relu(tf.nn.bias_add(_conv1,_b['bc1']))
    #池化层
    _pool1 = tf.nn.max_pool(_conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    #随机失活
    _pool_drl = tf.nn.dropout(_pool1,_keepratio)
    #conv layer2
    _conv2 = tf.nn.conv2d(_pool_drl,_w['wc2'],strides=[1,1,1,1],padding="SAME")
    #激活函数
    _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2,_b['bc2']))
    #池化层
    _pool2 = tf.nn.max_pool(_conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    #随机失活
    _pool_dr2 = tf.nn.dropout(_pool2,_keepratio)
    
    #全连接层
    _densel = tf.reshape(_pool_dr2,[-1,_w['wd1'].get_shape().as_list()[0]])
    #激活函数
    _fcl = tf.nn.relu(tf.add(tf.matmul(_densel,_w['wd1']),_b['bd1']))
    #随机失活
    _fc_dr1 = tf.nn.dropout(_fcl,_keepratio)
    _out = tf.add(tf.matmul(_fc_dr1,_w['wd2']),_b['bd2'])
    
    out = {
        'input_r':_input_r,'conv1':_conv1,'pool1':_pool1,
        'pool_dr1':_pool_drl,'conv2':_conv2,'pool2':_pool2,
        'pool_dr2':_pool_dr2,'dense1':_densel,'fc1':_fcl,
        'fc_dr1':_fc_dr1,'out':_out
    }
    
    return out
    

In [6]:
x = tf.placeholder(tf.float32,[None,784])
y= tf.placeholder(tf.float32,[None,10])
keepratio = tf.placeholder(tf.float32)

_pred = conv_basic(x,weights,biases,keepratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred,labels=y))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
_corr = tf.equal(tf.argmax(_pred,1),tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(_corr,tf.float32))

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

trian_epochs = 20
batch_size = 50
display_step = 1
for epoch in range(trian_epochs):
    avg_cost = 0
#     total_batch = int(mnist.train.num_examples/batch_size)
    total_batch = 10
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        sess.run(optm,feed_dict={x:batch_xs,y:batch_ys,keepratio:0.6})
        avg_cost += sess.run(cost,feed_dict={x:batch_xs,y:batch_ys,keepratio:0.6})
        
    if epoch % display_step ==0:
        print("Epoch: %03d/%03d cost: %.9f"%(epoch,trian_epochs,avg_cost))
        train_acc = sess.run(accr,feed_dict={x:batch_xs,y:batch_ys,keepratio:1.})
        print("Training accuracy: %.3f"%(train_acc))   
print("Finsh!")

Epoch: 000/020 cost: 183.615068436
Training accuracy: 0.220
Epoch: 001/020 cost: 57.183549404
Training accuracy: 0.520
Epoch: 002/020 cost: 28.334005356
Training accuracy: 0.660
Epoch: 003/020 cost: 19.346994281
Training accuracy: 0.640
Epoch: 004/020 cost: 15.639134824
Training accuracy: 0.680
Epoch: 005/020 cost: 13.976569653
Training accuracy: 0.720
Epoch: 006/020 cost: 13.146050096
Training accuracy: 0.840
Epoch: 007/020 cost: 11.411254466
Training accuracy: 0.880
Epoch: 008/020 cost: 10.884806156
Training accuracy: 0.880
Epoch: 009/020 cost: 10.261336207
Training accuracy: 0.920
Epoch: 010/020 cost: 9.416776359
Training accuracy: 0.920
Epoch: 011/020 cost: 8.749617279
Training accuracy: 0.840
Epoch: 012/020 cost: 6.961127371
Training accuracy: 0.940
Epoch: 013/020 cost: 7.402644455
Training accuracy: 0.880
Epoch: 014/020 cost: 7.346008480
Training accuracy: 0.880
Epoch: 015/020 cost: 5.873536199
Training accuracy: 0.980
Epoch: 016/020 cost: 7.291177958
Training accuracy: 0.940
Epo